In [ ]:
import delfi.generator as dg
import delfi.inference as infer
import delfi.utils.io as io
import model.utils as utils
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from model.HodgkinHuxley import HodgkinHuxley
from model.HodgkinHuxleyStatsMoments import HodgkinHuxleyStatsMoments
from run_genetic import run_deap

import sys; sys.path.append('../')
from common import col, svg, plot_pdf, samples_nd

%matplotlib inline

PANEL_A = 'svg/supp_ibea_a.svg'
PANEL_B = 'svg/supp_ibea_b.svg'
PANEL_B1 = 'svg/supp_ibea_b1.svg'
PANEL_C = 'svg/supp_ibea_c.svg'
PANEL_D = 'svg/supp_ibea_d.svg'


LABELS_HH =[r'$g_{Na}$', r'$g_{K}$', r'$g_{l}$', r'$g_{M}$', r'$\tau_{max}$',
            '-'+ r'$V_{T}$', r'$\sigma$', '-'+r'$E_{l}$']
LABELS_HH_SUMSTATS = [r'$sp$',r'$rpot$',r'$\sigma_{rpot}$','$m_1$','$m_2$','$m_3$','$m_4$']

## Set up the model

In [ ]:
seed = 1
seed_p = 2
n_xcorr = 0
n_mom = 4
n_summary = 7
summary_stats = 1
prior_uniform = True
prior_log = False
prior_extent = True
cython=True

true_params, labels_params = utils.obs_params(reduced_model=False)
I, t_on, t_off, dt = utils.syn_current()

# increase noise in true parameters
true_params[6] = .14

obs = utils.syn_obs_data(I, dt, true_params, seed=seed, cython=cython)
obs_stats = utils.syn_obs_stats(data=obs, I=I, t_on=t_on, t_off=t_off, dt=dt, params=true_params,
                                seed=seed, n_xcorr=n_xcorr, n_mom=n_mom, cython=cython,
                                summary_stats=summary_stats,n_summary=n_summary)

# define model, prior, summary statistics and generator
n_processes = 10

seeds_model = np.arange(1,n_processes+1,1)
m = []
for i in range(n_processes):
    m.append(HodgkinHuxley(I, dt, V0=obs['data'][0], reduced_model=False, seed=seeds_model[i],
                           cython=cython,prior_log=prior_log))
p = utils.prior(true_params=true_params,prior_uniform=prior_uniform,
                prior_extent=prior_extent,prior_log=prior_log, seed=seed_p)
s = HodgkinHuxleyStatsMoments(t_on=t_on, t_off=t_off, n_xcorr=n_xcorr, n_mom=n_mom, n_summary=n_summary)
g = dg.MPGenerator(models=m, prior=p, summary=s)
    
bounds = np.asarray([p.lower,p.upper]).T

## SNPE (if not run before)

In [ ]:
seed = 1
pilot_samples = 1000
n_sims = 100000
n_rounds = 1
n_components = 2
n_hiddens = [50]*2
n_xcorr = 0
n_mom = 4
n_summary = 7
summary_stats = 1
density = 'maf'  # 'mog' or 'maf'

res = infer.APT(g, 
    obs=obs_stats, 
    pilot_samples=pilot_samples, 
    n_hiddens=n_hiddens,
    seed=seed, 
    n_mades=5,
    prior_norm=True,
    impute_missing=False,
    density=density,
)

log, train_data, posterior = res.run(
    n_train=n_sims, 
    n_rounds=n_rounds, 
    minibatch=500,
    epochs=1000,
    silent_fail=False,
    proposal='mog',
    val_frac=0.1,
    monitor_every=1,
    verbose=True)

In [ ]:
# # load SNPE posterior
# filename1 = './results/posterior_7_single_round_maf_lfs.pkl'
# _, _, posterior = io.load_pkl(filename1)

## load results from pre-run genetic algorithm

In [ ]:
# to change size of hall-of-fame, change number of samples at
# https://github.com/BlueBrain/BluePyOpt/blob/master/bluepyopt/deapext/optimisations.py#L139

algo = 'ibea'
offspring_size = 1000
max_ngen = 100
run_ibea = 2 #2-6
final_pop, halloffame, log, hist = io.load_pkl('./results/'+\
                                               'sim_run_'+str(run_ibea)+'_offspr'+str(offspring_size)+\
                                               '_max_gen'+str(max_ngen)+ '_param8_7stats_' + algo + '.pkl')

In [ ]:
fitness_ibea1 = np.zeros(len(hist.genealogy_history))
all_params_ibea1 = []
for i in range(len(hist.genealogy_history)):
    fitness_ibea1[i] = hist.genealogy_history[i+1].fitness.sum
    all_params_ibea1.append(hist.genealogy_history[i+1])

# sort parameters across all generations according to fitness
fitness_ibea, ind_uni_fit_ibea = np.unique(fitness_ibea1, axis=0, return_index=True)
all_params_ibea = np.asarray(all_params_ibea1)[ind_uni_fit_ibea]

n_ibea = len(all_params_ibea)

## hall-of-fame solutions plotted with SNPE posterior

In [ ]:
# hall-of-fame parameters
n_params = len(bounds[:,0])
n_hof = len(halloffame)
params_hof = np.zeros((n_hof,n_params))
for i in range(n_hof):
    params_hof[i,:] = halloffame[i]

In [ ]:
# finding MAP (or close to) of SNPE posterior

samples_snpe_all1 = posterior[-1].gen(100000)
std_snpe = np.std(samples_snpe_all1,0)
ind_hi_prob_snpe1 = np.argsort(posterior[-1].eval(samples_snpe_all1),axis=0)[::-1]
samples_snpe = samples_snpe_all1[ind_hi_prob_snpe1[0:n_hof],:]

sample_snpe_max = samples_snpe_all1[ind_hi_prob_snpe1[0],:]


# evolutionary strategy
import cma

def mode_sample(posterior_):
#     init = true_params
    init = sample_snpe_max
    es = cma.CMAEvolutionStrategy(
        init,
        0.001,
        {'scaling_of_variables':(p.upper-p.lower)})
    es.optimize(lambda x: -1. * posterior_.eval(x) )
    es.result_pretty()
    return es.best.x

sample_snpe_max = mode_sample(posterior[-1])

In [ ]:
# plotting
fig_inches = (4.8, 4.8)
prior_lims = np.vstack((p.lower, p.upper)).T

all_params = [true_params]+list(params_hof)

col_ibea_rgb = (252/255,130/255,62/255)
all_cols = [col['GT']]+n_hof*[col_ibea_rgb]

MPL_RC = '../.matplotlibrc'
with mpl.rc_context(fname=MPL_RC):
    fig, axes = samples_nd(posterior[-1].gen(10000),
                               limits=prior_lims,
                               ticks=prior_lims,
                               labels=LABELS_HH,
                               fig_size=fig_inches,
                               title='',
                               diag='kde',
                               upper='kde',
                               hist_diag={'bins': 50},
                               hist_offdiag={'bins': 50},
                               kde_diag={'bins': 50, 'color': col['SNPE']},
                               kde_offdiag={'bins': 50},
                               points=all_params,
                               points_offdiag={'markersize': 3},
                               points_colors=all_cols);
    
    plt.savefig(PANEL_A, facecolor='None', transparent=True)
    plt.show()

## sets of N samples across all generations ordered according to IBEA objective plotted with SNPE posterior

In [ ]:
n_params_slice_ibea = 20000
num_slices = int(n_ibea/n_params_slice_ibea)

params_contour_plot = []
fitness_contour_plot = []
for i in range(num_slices):
    params_contour_plot.append(all_params_ibea[i*n_params_slice_ibea:(i+1)*n_params_slice_ibea,:])
    fitness_contour_plot.append(fitness_ibea[i*n_params_slice_ibea:(i+1)*n_params_slice_ibea])

params_contour_plot.reverse()
params_contour_plot.append(posterior[-1].gen(10000))
fitness_contour_plot.reverse()

In [ ]:
# plotting
fig_inches = (4.8, 4.8)
prior_lims = np.vstack((p.lower, p.upper)).T

# col IBEA slices
col_ibea = sns.light_palette(col_ibea_rgb, n_colors=num_slices).as_hex()
col_snpe = sns.light_palette(col['SNPE'], n_colors=2).as_hex()[1]
col_contrs = col_ibea+[col_snpe]

MPL_RC = '../.matplotlibrc'
with mpl.rc_context(fname=MPL_RC):
    fig, axes = samples_nd(params_contour_plot, 
                   samples_colors=col_contrs,
                   title='',
                   upper='contour', 
                   diag='kde',
                   limits=prior_lims,
                   ticks=prior_lims,
                   labels=LABELS_HH,
                   kde_offdiag={'bins': 100},
                   contour_offdiag={'levels': [0.68]},
                   fig_size=fig_inches,
                   points=[true_params],
                   points_offdiag={'markersize': 3},
                   points_colors=[col['GT']]);

    plt.savefig(PANEL_B, facecolor='None', transparent=True)
    plt.show()

In [ ]:
# colormap

MPL_RC = '../.matplotlibrc'
with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=(1,1))

    ax1 = fig.add_axes([0.05, 0.80, 0.1, 0.6])
    # ax1 = fig.add_axes([0.05, 0.80, 0.6, 0.1])
    # [left, bottom, width, height]

    cmap = sns.light_palette(col_ibea_rgb, n_colors=num_slices, as_cmap=True)

    cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap,orientation='vertical')
    # cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap,norm=norm,orientation='horizontal')
    # cb1.set_label('IBEA objective', x=1.25, labelpad=-13)
    cb1.outline.set_visible(False)
    cb1.set_ticks([0,0.5,1])
    cb1.ax.tick_params(size=0)
    cb1.set_ticklabels(['low','IBEA fitness','high'])
    
    plt.savefig(PANEL_B1, facecolor='None', transparent=True)
    plt.show()

## simulations for SNPE samples and hall-of-fame

In [ ]:
# simulations
t = obs['time']

obs_stats_norm = obs_stats/res.stats_std


# hall-of-fame
n_hof = len(halloffame)
V_hof = np.zeros((len(t),n_hof))
sumstats_hof = np.zeros((n_summary,n_hof))
for i in range(n_hof):
    x = m[0].gen_single(halloffame[i])
    V_hof[:,i] = x['data']
    sumstats_hof[:,i] = s.calc([x])
    
sumstats_hof_norm = sumstats_hof.T/res.stats_std


# SNPE samples
V_snpe = np.zeros((len(t),n_hof))
sumstats_snpe = np.zeros((n_summary,n_hof))
for i in range(n_hof):
    x = m[0].gen_single(samples_snpe[i,:])
    V_snpe[:,i] = x['data']
    sumstats_snpe[:,i] = s.calc([x])
    
sumstats_snpe_norm = sumstats_snpe.T/res.stats_std

In [ ]:
# plotting
col_ibea2 = sns.light_palette(col_ibea_rgb, n_colors=3).as_hex()

n_params = len(bounds[:,0])
n_summary_stats = len(obs_stats[0,:])
y_obs = obs['data']
duration = np.max(t)

xlim_int = [-10, duration]
ylim_int = [-100,60]

width = 0.3
n_colm = int(n_hof/2)

MPL_RC = '../.matplotlibrc'
with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=(5.,1.5))
    for i in range(n_colm):
        plt.subplot(2,n_colm,i+1)
        plt.plot(t, y_obs, color=col['GT'], lw=1.2, label='observation', clip_on=False, zorder=100)
        plt.plot(t, V_snpe[:,i], color=col['CONSISTENT1'], lw=1.2, label='SNPE', clip_on=False, zorder=100)
        ax = plt.gca()       
        ax.spines['bottom'].set_color('w')
        ax.spines['left'].set_color('w')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlim(xlim_int)
        ax.set_ylim(ylim_int)
        
        if i==int(n_colm/2):
            handles, labels = ax.get_legend_handles_labels()

        plt.subplot(2,n_colm,i+1+n_colm)
        plt.plot(t, y_obs, color=col['GT'], lw=1.2, clip_on=False, zorder=100)
        plt.plot(t, V_hof[:,i], color=col_ibea2[2], lw=1.2, label='hall-of-fame', clip_on=False, zorder=100)
        ax = plt.gca()       
        ax.spines['bottom'].set_color('w')
        ax.spines['left'].set_color('w')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlim(xlim_int)
        ax.set_ylim(ylim_int)
        
        if i==int(n_colm/2):
#             plt.legend(bbox_to_anchor=(1.1, -1.1), ncol=3)
            handles1, labels1 = ax.get_legend_handles_labels()
            fig.legend(handles+handles1, labels+labels1, bbox_to_anchor=(.84, 0.18), ncol=3)

        plt.tight_layout(h_pad=0.01, w_pad=0.01, pad=0.01)

        if i == 0:
            delta_bar_x = 50
            bar_x = [xlim_int[0],xlim_int[0]+delta_bar_x]
            delta_bar_y = 80
            bar_y = [ylim_int[0],ylim_int[0]+delta_bar_y]

            plt.plot(bar_x, [ylim_int[0], ylim_int[0]],'k',linewidth=1.5, clip_on=False, zorder=100)
            plt.xlabel(str(delta_bar_x)+' ms',x=.17)
            plt.plot([xlim_int[0], xlim_int[0]], bar_y,'k',linewidth=1.5, clip_on=False, zorder=100)
            plt.ylabel(str(delta_bar_y)+' mV',y=.30)
    
    plt.savefig(PANEL_C, facecolor='None', transparent=True)
    plt.show()

## multiple simulations for the same parameter sets

In [ ]:
# number of simulations for same parameter set
num_rep = 100

# hall-of-fame
V_hof_multi = np.zeros((len(t),num_rep))
sumstats_hof_multi = np.zeros((n_summary,num_rep))
for i in range(num_rep):
    x = m[0].gen_single(halloffame[0])
    V_hof_multi[:,i] = x['data']
    sumstats_hof_multi[:,i] = s.calc([x])
    
sumstats_hof_multi_norm = sumstats_hof_multi.T/res.stats_std


# SNPE samples
V_snpe_multi = np.zeros((len(t),num_rep))
sumstats_snpe_multi = np.zeros((n_summary,num_rep))
for i in range(num_rep):
    x = m[0].gen_single(sample_snpe_max)
    V_snpe_multi[:,i] = x['data']
    sumstats_snpe_multi[:,i] = s.calc([x])
    
sumstats_snpe_multi_norm = sumstats_snpe_multi.T/res.stats_std

In [ ]:
mn_sumstats_hof = np.mean(sumstats_hof_multi_norm,0)
err_sumstats_hof = np.std(sumstats_hof_multi_norm,0)

mn_sumstats_snpe = np.nanmean(sumstats_snpe_multi_norm,0)
err_sumstats_snpe = np.nanstd(sumstats_snpe_multi_norm,0)

MPL_RC = '../.matplotlibrc'
with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=(2.8,1.5))
    width = 0.3
    plt.bar(np.linspace(0,n_summary_stats-1,n_summary_stats),obs_stats_norm[0],
            width=width,color=col['GT'], edgecolor='w', label='observation')
    plt.bar(np.linspace(0,n_summary_stats-1,n_summary_stats)+width,mn_sumstats_snpe,
            yerr=[err_sumstats_snpe,err_sumstats_snpe],width=width,color=col['CONSISTENT1'],
            edgecolor='w', label='SNPE')
    plt.bar(np.linspace(0,n_summary_stats-1,n_summary_stats)+2*width,mn_sumstats_hof,
            yerr=[err_sumstats_hof,err_sumstats_hof],width=width,color=col_ibea2[2],
            edgecolor='w', label='hall-of-fame')
    ax = plt.gca()
    ax.set_xlim(-1.5*width,n_summary_stats+width/2)
    ax.set_xticks(np.linspace(0,n_summary_stats-1,n_summary_stats)+width)
    ax.set_xticklabels(LABELS_HH_SUMSTATS)
    plt.ylabel(r'$\frac{f}{\sigma_{f \ PRIOR}}$')

    ax.yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%g'))
    plt.ylim([-5., 1.])
    ax.set_yticks([-5, -2, 1])
        
    sns.despine(offset=10, trim=True)

    plt.savefig(PANEL_D, facecolor='None', transparent=True)
    plt.show()

## compose figure

In [ ]:
from svgutils.compose import *

# > Inkscape pixel is 1/90 of an inch, other software usually uses 1/72.
# > http://www.inkscapeforum.com/viewtopic.php?f=6&t=5964
svg_scale = 1.25  # set this to 1.25 for Inkscape, 1.0 otherwise

# Panel letters in Helvetica Neue, 12pt, Medium
kwargs_text = {'size': '12pt', 'font': 'Arial', 'weight': '800'}

f = Figure("20.3cm", "17.0cm",

    Panel(
          SVG(PANEL_A).scale(svg_scale).scale(0.95).move(0, 15),
          Text("a", 0, 13, **kwargs_text),
    ).move(0, 0),

    Panel(
          SVG(PANEL_B).scale(svg_scale).scale(0.95).move(0,15),
          SVG(PANEL_B1).scale(svg_scale).move(10,200),
          Text("b", 0, 13, **kwargs_text),
    ).move(370, 0),

    Panel(
          SVG(PANEL_C).scale(svg_scale).move(0, 15),
          Text("c", 0, 13, **kwargs_text),
    ).move(0, 410),

    Panel(
        SVG(PANEL_D).scale(svg_scale).move(0, 15),
        Text("d", 0, 13, **kwargs_text),
    ).move(470, 410),

)

!mkdir -p fig
f.save("fig/fig5_hh_supp_ibea.svg")
svg('fig/fig5_hh_supp_ibea.svg')

### export to pdf and png

In [ ]:
!inkscape --export-pdf fig/fig5_hh_supp_ibea.pdf fig/fig5_hh_supp_ibea.svg
!inkscape --export-png fig/fig5_hh_supp_ibea.png fig/fig5_hh_supp_ibea.svg